<a href="https://colab.research.google.com/github/phionex2/Deep-Learning/blob/main/keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd


In [19]:
df = pd.read_csv('diabetes.csv')

In [20]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [21]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [22]:
X = df.iloc[:,:-1].values
y =df.iloc[:,-1].values

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [24]:
X = scaler.fit_transform(X)#Scale the x data

In [25]:
X.shape

(768, 8)

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 1)

In [27]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [28]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [29]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 17ms/step - loss: 0.7325 - accuracy: 0.5391 - val_loss: 0.7154 - val_accuracy: 0.5455
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6722 - accuracy: 0.6059 - val_loss: 0.6572 - val_accuracy: 0.6039
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6287 - accuracy: 0.6450 - val_loss: 0.6153 - val_accuracy: 0.6688
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5959 - accuracy: 0.6726 - val_loss: 0.5862 - val_accuracy: 0.6948
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5711 - accuracy: 0.6954 - val_loss: 0.5621 - val_accuracy: 0.7143
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5512 - accuracy: 0.7020 - val_loss: 0.5434 - val_accuracy: 0.7273
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5350 - accuracy: 0.7231 - val_loss: 0.5289 - val_accuracy: 0.7468
Epoch 8/100


In [30]:
#how to select appropriate optimizer
!pip install -U keras-tuner

In [31]:
import kerastuner as kt

In [32]:
def build_model(hp):       #hp-Hyperparameter
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [33]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [34]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 05s


In [35]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [36]:
model = tuner.get_best_models(num_models=1)[0]

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=4,validation_data=(X_test,y_test))

Epoch 5/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5483 - accuracy: 0.7248 - val_loss: 0.5240 - val_accuracy: 0.7792
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5293 - accuracy: 0.7329 - val_loss: 0.5073 - val_accuracy: 0.7727
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5143 - accuracy: 0.7427 - val_loss: 0.4948 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5028 - accuracy: 0.7590 - val_loss: 0.4855 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4926 - accuracy: 0.7541 - val_loss: 0.4772 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4842 - accuracy: 0.7622 - val_loss: 0.4709 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4779 - accuracy: 0.7606 - val_loss: 0.4659 - val_accuracy: 0.8052
Epoch 12/1

# No. of neurons in layers

In [39]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [40]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir')

In [41]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.6038960814476013

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 14s


In [42]:
tuner.get_best_hyperparameters()[0].values

{'units': 24}

In [43]:
model = tuner.get_best_models(num_models=1)[0]

In [44]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6238 - accuracy: 0.6775
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6097 - accuracy: 0.6857
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5982 - accuracy: 0.6873
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5885 - accuracy: 0.6922
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5795 - accuracy: 0.7020
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5719 - accuracy: 0.7052
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5652 - accuracy: 0.7020
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5590 - accuracy: 0.7036
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5534 - accuracy: 0.7036
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5482 - accuracy: 0.7068
Epo

# No to choose no. of layers

In [45]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [46]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='num_layers')

In [47]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 08s


In [48]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [49]:
model = tuner.get_best_models(num_models=1)[0]

In [50]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4683 - accuracy: 0.7655 - val_loss: 0.5014 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4571 - accuracy: 0.7883 - val_loss: 0.4714 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4513 - accuracy: 0.7883 - val_loss: 0.4715 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4419 - accuracy: 0.7964 - val_loss: 0.4690 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4326 - accuracy: 0.8029 - val_loss: 0.4729 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4156 - accuracy: 0.7997 - val_loss: 0.4887 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4111 - accuracy: 0.8192 - val_loss: 0.4672 - val_accuracy: 0.7922
Epoch 14